In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

A Little note before the peer review:
My aim for this was initially just houses >500k would be 
expensive = 1 and less than would be affordable =0. Instead of going this classification route I decided to specify a high value if their price per sqft > median price per sqft. There was another way I wanted to experiment with, which I will see too maybe in another notebook, but for this peer review I wanted to see how this method and line of thought worked out. 

In [9]:
austin_df = pd.read_csv("austinHousingData.csv")
austin_df.columns = austin_df.columns.str.strip()

In [10]:
austin_df = austin_df[(austin_df["latestPrice"] > 0) & (austin_df["livingAreaSqFt"] > 0)]
austin_df["price_per_sqft"] = austin_df["latestPrice"] / austin_df["livingAreaSqFt"]
median_price_per_sqft = austin_df["price_per_sqft"].median()
austin_df["is_high_value"] = (austin_df["price_per_sqft"] > median_price_per_sqft).astype(int)


In [11]:
numeric_df = austin_df.select_dtypes(include=[np.number])
cols_to_drop = [col for col in ['zpid', 'price_per_sqft', 'is_high_value', 'latestPrice'] if col in numeric_df.columns]
numeric_features = numeric_df.drop(columns=cols_to_drop)

In [12]:
model_df = pd.concat([numeric_features, austin_df["is_high_value"]], axis=1).dropna()


In [13]:
X = model_df.drop(columns=["is_high_value"])
y = model_df["is_high_value"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_scaled, y_train)
y_pred = logreg.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76      1517
           1       0.76      0.74      0.75      1518

    accuracy                           0.75      3035
   macro avg       0.75      0.75      0.75      3035
weighted avg       0.75      0.75      0.75      3035



link for the dataset can be found on kaggle here: https://www.kaggle.com/datasets/ericpierce/austinhousingprices/data

One last thing to note. I vastly prefer the findings here than in another way I was contemplating classification here which was through defining luxury based on rules like num of bathrooms and garage spaces and school rating. Since these things are highly subjective it is hard to justify cutoff rules like in comparing a house with more bathrooms to ones with less but has a pool. This I have found lead to class imbalances. The "hypothesis" I tested above with price/sqft is a clean way to classify real estate models. It better differentiates value for money especially in a context where pricing for an apartment vs house is very different. 